<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Send contributor activity on slack
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Reopen_issue.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Reopen+issue:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #github #activity #update #api #snippet #operations #slack 

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Description:** This notebook explains how to send the GitHub activity on Slack

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)

## Input

### Import libraries

In [ ]:
import os
import json
from datetime import datetime
from github import Github
import naas
import pandas as pd
import requests
from naas_drivers import slack

### Setup Variables
- `token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `owner`: owner of the repository
- `repo`: name of the repository
- `issue_number`: number of the issue

In [ ]:
# Input variables
github_token = naas.secret.get("GITHUB_TOKEN") or "GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks" #example
contributor_profile = "Benjifilly"
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "SLACK_TOKEN"
slack_channel = "naas-templates-contributors"

## Model

### Get Closed PRs
- Filter on contributor
- Get files added on PR (GitHub_Get_files_added_on_pull_request.ipynb)

### Get PRs

In [ ]:
# Connect to the GitHub API
g = Github(github_token)
# Get the repository
repo = g.get_repo(repo_name)
# Get the closed PR
pull_requests = repo.get_pulls(state="open")

# Print the closed PR
print("✅ Pull Requests fetched:", pull_requests.totalCount)
data = []
for index, pr in enumerate(pull_requests):
    # Init
    assignee_login = None
    assignee = pr.raw_data.get("assignee")
    if assignee:
        assignee_login = assignee.get("login")
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.raw_data.get("url"),
        "assignee": assignee_login,
        "created_at": pr.raw_data.get("created_at"),
        "updated_at": pr.raw_data.get("updated_at"),
    }
    data.append(tmp)

df_pr = pd.DataFrame(data)
df_pr.head(3)

### Filter dataframe on contributor
Do not forget to use .reset_index(drop=True)

In [ ]:
contributor_filtered_df = df_pr[df_pr['assignee'] == contributor_profile].reset_index(drop=True)
print("✅ PR Opened:", len(contributor_filtered_df))
contributor_filtered_df.head(1)

### Get PR in Review and WIP

Find if "https://github.com/jupyter-naas/awesome-notebooks/issues/" in comments or PR description

In [ ]:
# The function "pd.set_option('display.max_colwidth', None)" is used in pandas, a data manipulation library in Python, to set the maximum width for displaying column contents in a tabular format to be unlimited, allowing the full content of each column to be displayed without truncation.
pd.set_option('display.max_colwidth', None)

# Init - Create empty list
prs_in_review = []
prs_work_in_progress = []

# Loop in df and last message on each PR
for row in contributor_filtered_df.itertuples():
    # Display PR number
    pr_number = row.number
    print(pr.body)
    
    # Get PR object
    pr = next((pr for pr in pull_requests if pr.number == pr_number), None)
    
    # If PR exists then get comments
    if pr:
        # Get comments from PR
        comments = pr.get_issue_comments()
        
        # Check if nb comments > 0 else PR status = WIP
        if comments.totalCount > 0:
            last_comment = comments[comments.totalCount-1].body.lower()
            
            # Check if "read to review" else PR status = WIP
            if "ready to review" in last_comment:
                prs_in_review.append(pr)
            else:
                prs_work_in_progress.append(pr)
        else:
            prs_work_in_progress.append(pr)
            
print("✅ Pull Requests in Review:", len(prs_in_review))
print("👨‍💻 Pull Requests Work in Progress:", len(prs_work_in_progress))

### Get issues

In [ ]:
new_issues = []
num_new_issues = 0
num_connected_issues = 0

try:
    # Get the repository
    repository = g.get_repo(f'{repo_name}')

    # Get issues assigned to the contributor
    issues = repository.get_issues(assignee=contributor_profile)

    new_issues = []
    num_new_issues = 0
    num_connected_issues = 0
    pr_numbers = set()

    # Get PR numbers from PR titles
    pull_requests = repository.get_pulls(state="open")
    for pr in pull_requests:
        pr_numbers.add(pr.number)

    for issue in issues:
        # Check if it is an issue and not a pull request
        if not issue.pull_request:
            issue_number = issue.number
            issue_title = issue.title
            issue_link = issue.html_url
            events = issue.get_events()

            # Check if any event has a status of "connected"
            connected_event = next(
                (event for event in events if event.event == 'connected'),
                None
            )

            if connected_event:
                num_connected_issues += 1
            else:
                # Check if issue number is mentioned in PR titles
                found_in_pr = any(str(issue_number) in pr.title for pr in pull_requests)
                if not found_in_pr:
                    num_new_issues += 1
                    new_issues.append(issue)


except Exception as e:
    print(f"An error occurred: {e}")

print(f"✅ Number of issues with PR linked: {num_connected_issues}")
print(f"❌ Number of issues without PR linked: {num_new_issues}")

In [ ]:
import re
new_issues = []
num_new_issues = 0
num_connected_issues = 0

try:
    # Get the repository
    repository = g.get_repo(f'{repo_name}')

    # Get issues assigned to the contributor
    issues = repository.get_issues(assignee=contributor_profile)

    for issue in issues:
        # Check if it is an issue and not a pull request
        if not issue.pull_request:
            issue_number = issue.number
            issue_title = issue.title
            issue_link = issue.html_url
            issue_description = issue.body

            # Check if any event has a status of "connected"
            events = issue.get_events()
            connected_event = next(
                (event for event in events if event.event == 'connected'),
                None
            )

            # Check if issue description contains "May be fixed by #<four-digit-number>"
            pr_linked_match = re.search(r"May be fixed by #(\d{4})", issue_description)

            if connected_event or pr_linked_match:
                num_connected_issues += 1
            else:
                num_new_issues += 1
                new_issues.append(issue)

except Exception as e:
    print(f"An error occurred: {e}")

print(f"✅ Number of issues with PR linked: {num_connected_issues}")
print(f"❌ Number of issues without PR linked: {num_new_issues}")

### Create Slack block

In [ ]:
prs_in_review.sort(key=lambda pr: pr.number, reverse=True)
prs_work_in_progress.sort(key=lambda pr: pr.number, reverse=True)
new_issues.sort(key=lambda issue: issue.number, reverse=True)
current_date = datetime.now().strftime("%B %d, %Y %H:%M")

blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{contributor_profile} - Activity update as of {current_date}",
            "emoji": True
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"✅ *List of PR to be validated ({len(prs_in_review)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|[{pr.number}] {pr.title}>"
            }
        }
        for pr in prs_in_review
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"👨‍💻 *List of current PR ({len(prs_work_in_progress)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|[{pr.number}] {pr.title}>"
            }
        }
        for pr in prs_work_in_progress
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f":bulb: *List of new issues ({len(new_issues)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{issue.html_url}|[{issue.number}] {issue.title}>"
            }
        }
        for issue in new_issues
    ]
]

## Output

### Send message on Slack

In [ ]:
slack.connect(slack_bot_token).send(slack_channel, text=None, blocks=blocks)